In [0]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib

In [2]:
data = pd.read_csv('CGMData.csv').T
data = data.reset_index(drop=True).fillna(data.mean());
# data = data.dropna(how='any',axis =0);
data

,0
0,136.0
1,135.0
2,136.0
3,135.0
4,134.0
...,...
44840,101.0
44841,98.0
44842,95.0
44843,91.0


In [12]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [0]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:590227.5353242328


In [5]:
# mod = sm.tsa.statespace.SARIMAX(data,
#                                 order=(1, 1, 1),
#                                 seasonal_order=(1, 0, 1, 12),
#                                 enforce_stationarity=False,
#                                 enforce_invertibility=False)
# results = mod.fit()
# print(results.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7171      0.003    222.483      0.000       0.711       0.723
ma.L1         -0.2675      0.003    -82.362      0.000      -0.274      -0.261
ar.S.L12       0.6809      0.041     16.710      0.000       0.601       0.761
ma.S.L12      -0.7106      0.040    -17.839      0.000      -0.789      -0.633
sigma2        36.9423      0.030   1241.079      0.000      36.884      37.001


In [30]:
y_pred = results.predict(start=len(data), end=len(data)+12)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/base/tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [0]:
def sarima_forecast(data):
    model = sm.tsa.statespace.SARIMAX(data,
                                order=(1, 1, 1),
                                seasonal_order=(1, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False).fit();
    y_pred = model.predict(len(data),len(data));
    return y_pred[0];

In [0]:
def train_test_split(data,n_test):
    return data[:n_test],data[n_test:];

In [0]:
def measure_error(actual,predicted):
    return sqrt(mean_squared_error(actual, predicted));

In [0]:
def walk_forward_validation(data,n_test):
    predictions = [];
    train,test = train_test_split(data,n_test);
    history = [x for x in train]
    y_pred = sarima_forecast(train);
    print (y_pred);
    for i in range(len(test)):
        y_pred = sarima_forecast(train)
        predictions.append(y_pred);
        history.append(y_pred);
    error= measure_error(test,predictions)
    return error;

In [7]:
walk_forward_validation(data,1000)

KeyError: ignored

44845    87.440294
dtype: float64